[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/02-langsmith.ipynb)

#### LangChain Essentials Course

# LangSmith Starter

LangSmith is a built-in observability service and platform that integrates _very easily_ with LangChain. You don't _need_ to use LangSmith for this course, but it can be very helpful in understanding what is happening, _and_ we recommend using it beyond this course for general development with LangChain — with all of that in mind we would recommend spending a little bit of time to get familiar with LangSmith.

## Setting up LangSmith

LangSmith does require an API key, but it comes with a generous free tier. You can sign up an account and get your API key [here](https://smith.langchain.com).

When using LangSmith, we need to setup our environment variables _and_ provide our API key, like so:

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env file

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "Abhilasha-Test-Project"
print(os.getenv("LANGCHAIN_API_KEY"))

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


In most cases, this is all we need to start seeing logs and traces in the [LangSmith UI](https://smith.langchain.com). By default, LangChain will trace LLM calls, chains, etc. We'll take a look at a quick example of this below.

## Default Tracing

As mentioned, LangSmith traces a lot of data without us needing to do anything. Let's see how that looks. We'll start by initializing our LLM. Again, this will need an [API key](https://platform.openai.com/api-keys).

In [2]:
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter Gemini API Key: "
)

gemini_model = "gemini-2.5-flash-lite"  # or "gemini-2.5-flash" for more power

from langchain_google_genai import ChatGoogleGenerativeAI

# For normal accurate responses
llm = ChatGoogleGenerativeAI(temperature=0.0, 
                             model=gemini_model, 
                             google_api_key=os.environ["GOOGLE_API_KEY"],
                             max_retries=2,
                             timeout=None,
                             max_tokens=None)

# For unique creative responses
creative_llm = ChatGoogleGenerativeAI(temperature=0.9, 
                                      model=gemini_model, 
                                      google_api_key=os.environ["GOOGLE_API_KEY"],
                                      max_retries=2,
                                      timeout=None,
                                      max_tokens=None)

Let's invoke our LLM and then see what happens in the LangSmith UI.

In [3]:
llm.invoke("Hii this side Abhilasha ! what can you do for me?")

AIMessage(content='Hi Abhilasha! It\'s great to meet you. I\'m a large language model, trained by Google.\n\nThink of me as a versatile AI assistant. I can help you with a wide range of tasks, including:\n\n**Information & Knowledge:**\n\n*   **Answering questions:** I can access and process information from the real world and provide you with answers on almost any topic.\n*   **Explaining concepts:** If you\'re struggling to understand something, I can break it down into simpler terms.\n*   **Summarizing text:** I can condense long articles, documents, or web pages into concise summaries.\n*   **Researching topics:** I can help you gather information on specific subjects.\n\n**Creative & Writing Tasks:**\n\n*   **Writing different kinds of creative content:** This includes poems, code, scripts, musical pieces, email, letters, etc.\n*   **Generating ideas:** If you\'re stuck for inspiration, I can brainstorm ideas for stories, projects, or anything else.\n*   **Drafting emails and lett

After this we should see that a new project (`aurelioai-langchain-course-langsmith-openai`) has been created in the LangSmith UI. Inside that project, we should see the trace from our LLM call:


![LangSmith LLM Trace](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-01.jpg?raw=1)

By default, LangSmith will capture plenty — however, it won't capture functions from outside of LangChain. Let's see how we can trace those.

## Tracing Non-LangChain Code

LangSmith can trace functions that are not part of LangChain, we just need to add the `@traceable` decorator. Let's try this for a few simple functions.

In [6]:
from langsmith import traceable
import random
import time


@traceable
def generate_random_number():
    return random.randint(0, 100)

@traceable
def generate_string_delay(input_str: str):
    number = random.randint(1, 5)
    time.sleep(number)
    return f"{input_str} ({number})"

@traceable
def random_error():
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "No error"

Let's run these a few times and see what happens.

In [7]:
from tqdm.auto import tqdm

for _ in tqdm(range(10)):
    generate_random_number()
    generate_string_delay("Hello")
    try:
        random_error()
    except ValueError:
        pass

/home/anushkaarora/Desktop/LangchainLearning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 10/10 [00:27<00:00,  2.73s/it]


Those traces should now be visible in the LangSmith UI, again under the same project:

![LangSmith Traces](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-02.jpg?raw=1)

We can various metrics here for each run. First, ofcourse, the run name. We can see any inputs and outputs from each run, we can see if the run raised any errors, it's start time, and latency. Inside the UI we can also filter for specific runs, like so:

![LangSmith UI filters](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-03.jpg?raw=1)

There are various other things we can do within the UI, but we'll leave that for you to explore.

Finally, we can also modify our traceable names if we'd like to make them more readable inside the UI. For example:

In [10]:
from langsmith import traceable

@traceable(name="Chitchat Maker")
def error_generation_function(question: str):
    delay = random.randint(0, 3)
    time.sleep(delay)
    number = random.randint(0, 1)
    if number == 0:
        raise ValueError("Random error")
    else:
        return "I'm great how are you?"

Let's run this a few times and see what we get in LangSmith.

In [11]:
for _ in tqdm(range(10)):
    try:
        error_generation_function("How are you today?")
    except ValueError:
        pass

100%|██████████| 10/10 [00:20<00:00,  2.01s/it]


Let's filter for the `Chitchat Maker` traceable and see our results.

![LangSmith Chitchat Maker runs](https://github.com/aurelio-labs/langchain-course/blob/main/assets/langsmith-ui-04.jpg?raw=1)

We can see our runs and their related metadata! That's it for this intro to LangSmith. As we work through the course we will (optionally) refer to LangSmith for digging into our runs.